### Transformation to a Scheme Register Machine

We will start with the standard recursive definition for factorial:

In [ ]:
(define fact
  (lambda (n)
    (if (= n 0)
        1
        (* n (fact (- n 1))))))

In [ ]:
(fact 5)

Next, we rewrite `fact` in continuation-passing style (CPS):

In [ ]:
(define fact-cps
  (lambda (n k)
    (if (= n 0)
        (k 1)
        (fact-cps (- n 1)
          (lambda (value)
            (k (* n value)))))))

To run `fact-cps` we need to provide the *initial continuation* `(lambda (value) value)`

In [ ]:
(fact-cps 5 (lambda (value) value))

For convenience, our top-level `fact` function will just call `fact-cps` with the initial continuation:

In [ ]:
(define fact
  (lambda (n)
    (fact-cps n (lambda (value) value))))

In [ ]:
(fact 5)

Notice that our continuations are represented as anonymous *lambda* functions. Here we are taking advantage of Scheme's ability to easily create and pass around higher-order functions. In the above code, there are two such continuation functions: `(lambda (value) value)` and `(lambda (value) (k (* n value)))`.  In general, a *lambda* expression that represents a continuation may or may not contain **free variables**, depending on the context in which it arises.  For example, the continuation `(lambda (value) value)` has no free variables, since `value` is bound by the lambda's formal parameter, while the continuation `(lambda (value) (k (* n value)))` has two free variables: `k` and `n`.

#### Changing the representation of continuations

We now change the representation of continuations from Scheme functions to Scheme data structures, which will just be ordinary lists (also referred to as **continuation records**). Each lambda expression that represents a continuation will get replaced by a call of the form `(make-cont <label> x y z ...)`, where `<label>` is a unique identifier for the continuation being replaced, and <tt>x y z ...</tt> are the free variables (if any) of the lambda expression. Here are the new definitions of `fact` and `fact-cps`, where the `(lambda (value) value)` continuation has been labeled `<cont-1>` and the `(lambda (value) (k (* n value)))` continuation has been labeled `<cont-2>`:

In [ ]:
(define fact
  (lambda (n)
    (fact-cps n (make-cont '<cont-1>))))

(define fact-cps
  (lambda (n k)
    (if (= n 0)
        (k 1)
        (fact-cps (- n 1) (make-cont '<cont-2> n k)))))

The `make-cont` function simply packages the `<cont-N>` label along with the values of any free variables *x, y, z, etc.* from the original lambda expression into a Scheme list of the form `(continuation <cont-N> x y z ...)`.

In [ ]:
(define make-cont
  (lambda args
    (cons 'continuation args)))

Here are some examples of continuation records:

In [ ]:
(make-cont '<cont-1>)

In [ ]:
(make-cont '<cont-2> 'free1 'free2)

In [ ]:
(make-cont '<cont-42> 10 20 30 40)

To make this code actually work, we need to define a new function `apply-cont` which takes a continuation *k* represented in this new form (a Scheme list), along with a *value*, and does the same thing to *value* as the previous functional representation of the continuation did, according to the continuation type specified by the label. Any free variables that are needed are just retrieved from the fields of the continuation record.

We also need to change all applications of the form `(k value)` to `(apply-cont k value)` since *k* is no longer represented as a Scheme function. Here is the resulting code:

In [1]:
(define fact
  (lambda (n)
    (fact-cps n (make-cont '<cont-1>))))

(define fact-cps
  (lambda (n k)
    (if (= n 0)
        (apply-cont k 1)
        (fact-cps (- n 1) (make-cont '<cont-2> n k)))))

(define apply-cont
  (lambda (k value)
    ;; k is now a list of the form (continuation <label> x y z ...)
    (let ((label (cadr k))
          (fields (cddr k))) ;; saved values of free variables
      (cond
       ((eq? label '<cont-1>) value)
       ((eq? label '<cont-2>)
        (let ((n (car fields))
              (k (cadr fields)))
          (apply-cont k (* n value))))
       (else (error "invalid continuation label"))))))

In [ ]:
(fact 5)

This code works, but notice that whenever `apply-cont` receives a continuation *k* to apply to some *value*, it uses a `cond` to decide what to do, based on *k*'s label. In this example, there are only two possible continuation labels, but larger programs could have many more. To make the code more efficient, we define separate functions corresponding to each label, and then simply apply the appropriate label function to *value* and the saved free variables. Instead of the labels being Scheme symbols, they are now Scheme functions that can be called directly.

In [ ]:
(define apply-cont
  (lambda (k value)
    (let ((label (cadr k))
          (fields (cddr k)))
      (label value fields))))

(define <cont-1>
  (lambda (value fields)
    value))

(define <cont-2>
  (lambda (value fields)
    (let ((n (car fields))
          (k (cadr fields)))
      (apply-cont k (* n value)))))

(define fact
  (lambda (n)
    (fact-cps n (make-cont <cont-1>)))) ;; removed ' from <cont-1>

(define fact-cps
  (lambda (n k)
    (if (= n 0)
        (apply-cont k 1)
        (fact-cps (- n 1) (make-cont <cont-2> n k))))) ;; removed ' from <cont-2>

In [ ]:
(fact 5)

In [ ]:
(make-cont <cont-1>)

In [ ]:
(make-cont <cont-2> 'free1 'free2)

#### Removing the reliance on the recursion stack

We have now changed the representation of continuations from Scheme functions to data structures, so our `fact-cps` function could in principle be implemented easily in languages that do not support higher-order functions. However, there is another feature of Scheme that we are implicitly relying on, namely the fact that Scheme imposes no limit on the depth of its recursion stack, which enables us to call `fact` with arbitrarily large values of *n*.

Unfortunately, many other languages impose a maximum depth on the recursion stack, which would cause a recursive factorial program to crash for sufficiently large values of *n*. Thus we need to further transform our code to avoid growing the recursion stack. We can accomplish this by passing information to functions through a set of global **registers** rather than via function arguments. Each function call of the form `(func arg1 arg2 arg3 ...)` will be replaced by a call of the form`(func)`, which in most other languages can be simulated by a simple "goto" instruction that does not grow the stack. The appropriate registers will be initialized to *arg1*, *arg2*, *arg3*, etc., prior to calling `(func)`.

#### Passing information via registers

We first define a set of registers to be used for passing information to functions in place of formal parameters. The registers needed by a particular function are determined by that function's formal parameters. For example, here are the `fact-cps` function and the top-level `fact` function, which calls `fact-cps` with *n* and the initial <tt>&lt;cont-1&gt;</tt> continuation:

In [ ]:
(define fact-cps
  (lambda (n k)
    (if (= n 0)
        (apply-cont k 1)
        (fact-cps (- n 1) (make-cont <cont-2> n k)))))

(define fact
  (lambda (n)
    (fact-cps n (make-cont <cont-1>))))

We will rewrite `fact-cps` as a function of no arguments, and create new registers `n_reg` and `k_reg` for passing the necessary information to `fact-cps`. In addition, wherever we call `(fact-cps)`, we must first assign the appropriate values to the registers. We also must change all references to the formal parameters <tt>n</tt> and <tt>k</tt> in `fact-cps` to <tt>n_reg</tt> and <tt>k_reg</tt>, respectively:

In [ ]:
(define n_reg 'undefined)
(define k_reg 'undefined)

(define fact-cps
  (lambda ()
    (if (= n_reg 0)
        (apply-cont k_reg 1)
        (begin
          ;; order of assignments matters!
          (set! k_reg (make-cont <cont-2> n_reg k_reg))
          (set! n_reg (- n_reg 1))
          (fact-cps)))))

(define fact
  (lambda (n)
    (set! k_reg (make-cont <cont-1>))
    (set! n_reg n)
    (fact-cps)))

In [ ]:
(fact 5)

Notice that in the definition of `fact-cps` above, the order of the assignment statements matters. The <tt>n_reg</tt> assignment must happen *after* the <tt>k_reg</tt> assignment, otherwise the value of <tt>n_reg</tt> saved in the <tt>&lt;cont-2&gt;</tt> continuation record will be incorrect.

We also need to transform `apply-cont`, `<cont-1>`, and `<cont-2>` in a similar way. Here are their current definitions:

In [ ]:
(define apply-cont
  (lambda (k value)
    (let ((label (cadr k))
          (fields (cddr k)))
      (label value fields))))

(define <cont-1>
  (lambda (value fields)
    value))

(define <cont-2>
  (lambda (value fields)
    (let ((n (car fields))
          (k (cadr fields)))
      (apply-cont k (* n value)))))

We will use the registers `k_reg` and `value_reg` to pass information to `apply-cont`. To pass information to the continuation label functions `<cont-1>` and `<cont-2>`, we will use the registers `value_reg` and `fields_reg`:

In [ ]:
;; additional registers
(define value_reg 'undefined)
(define fields_reg 'undefined)

(define apply-cont
  (lambda ()
    (let ((label (cadr k_reg))
          (fields (cddr k_reg)))
      ;; set up value_reg and fields_reg before calling (label)
      (set! value_reg value_reg)
      (set! fields_reg fields)
      (label))))

(define <cont-1>
  (lambda ()
    value_reg))

(define <cont-2>
  (lambda ()
    (let ((n (car fields_reg))
          (k (cadr fields_reg)))
      ;; set up k_reg and value_reg before calling (apply-cont)
      (set! k_reg k)
      (set! value_reg (* n value_reg))
      (apply-cont))))

We also need to rewrite the call to `apply-cont` that appears in the definition of `fact-cps`:

In [ ]:
(define fact-cps
  (lambda ()
    (if (= n_reg 0)
        (begin
          ;; set up k_reg and value_reg before calling (apply-cont)
          (set! k_reg k_reg)
          (set! value_reg 1)
          (apply-cont))
        (begin
          (set! k_reg (make-cont <cont-2> n_reg k_reg))
          (set! n_reg (- n_reg 1))
          (fact-cps)))))

In [ ]:
(fact 5)

Notice that the assignment statements `(set! k_reg k_reg)` and `(set! value_reg value_reg)` in `fact-cps` and `apply-cont` are unnecessary, so we can simply remove them from the code.

In [ ]:
(define fact-cps
  (lambda ()
    (if (= n_reg 0)
        (begin
          (set! value_reg 1)
          (apply-cont))
        (begin
          (set! k_reg (make-cont <cont-2> n_reg k_reg))
          (set! n_reg (- n_reg 1))
          (fact-cps)))))

(define apply-cont
  (lambda ()
    (let ((label (cadr k_reg))
          (fields (cddr k_reg)))
      (set! fields_reg fields)
      (label))))

In [ ]:
(fact 5)

#### The trampoline

Although the above code works, it still generates an arbitrarily long chain of tail-recursive function calls of the form `(fact-cps)`, `(apply-cont)`, `(label)`, etc., each of which essentially acts like a "goto" instruction. This is not a problem in Scheme, since no limit is imposed on the length of such call-chains. However, in other languages it could be a problem. Therefore we need to break the chain of function calls into single steps. This is accomplished through the use of a **trampoline**, which is essentially a while-loop that performs the computation one step at a time and avoids building up a chain of function calls.

The trampoline uses a special register called `pc`, which contains the next function to call on each loop cycle. Calling the function simply updates the registers appropriately for the next loop cycle. The `pc` register itself also gets updated on each cycle. This process continues until the `pc` register becomes empty, at which point the final result of the computation will be available in the register `final_reg`.

In [ ]:
;; additional registers
(define pc 'undefined)
(define final_reg 'undefined)

;; equivalent to a while-loop
(define trampoline
  (lambda ()
    (if pc
        (begin
          (pc)
          (trampoline))
        final_reg)))

Instead of calling a function directly such as `(fact-cps)`, we replace the function call with `(set! pc fact-cps)`, which sets the `pc` register to the `fact-cps` function itself. The trampoline will then invoke it within the loop. All functions other than the trampoline simply execute if-statements and assignments, without ever calling another function directly. Here are the transformed versions of the other functions, showing the changes made to the code:

In [2]:
(define fact-cps
  (lambda ()
    (if (= n_reg 0)
        (begin
          (set! value_reg 1)
          (set! pc apply-cont))   ;; changed
        (begin
          (set! k_reg (make-cont <cont-2> n_reg k_reg))
          (set! n_reg (- n_reg 1))
          (set! pc fact-cps)))))  ;; changed

(define apply-cont
  (lambda ()
    (let ((label (cadr k_reg))
          (fields (cddr k_reg)))
      (set! fields_reg fields)
      (set! pc label))))          ;; changed

(define <cont-1>
  (lambda ()
    (set! final_reg value_reg)    ;; changed
    (set! pc #f)))                ;; added

(define <cont-2>
  (lambda ()
    (let ((n (car fields_reg))
          (k (cadr fields_reg)))
      (set! k_reg k)
      (set! value_reg (* n value_reg))
      (set! pc apply-cont))))     ;; changed

The top-level function `fact` initializes the registers and then starts the trampoline, which runs the computation to completion.

In [ ]:
(define fact
  (lambda (n)
    (set! k_reg (make-cont <cont-1>))
    (set! n_reg n)
    (set! pc fact-cps)
    (trampoline)))

In [ ]:
(fact 5)

#### The register machine

The complete register machine code for factorial is given below:

In [ ]:
;; global registers
(define n_reg 'undefined)
(define k_reg 'undefined)
(define value_reg 'undefined)
(define fields_reg 'undefined)
(define pc 'undefined)
(define final_reg 'undefined)

(define trampoline
  (lambda ()
    (if pc
        (begin
          (pc)
          (trampoline))
        final_reg)))

(define make-cont
  (lambda args
    (cons 'continuation args)))

(define apply-cont
  (lambda ()
    (let ((label (cadr k_reg))
          (fields (cddr k_reg)))
      (set! fields_reg fields)
      (set! pc label))))

(define <cont-1>
  (lambda ()
    (set! final_reg value_reg)
    (set! pc #f)))

(define <cont-2>
  (lambda ()
    (let ((n (car fields_reg))
          (k (cadr fields_reg)))
      (set! k_reg k)
      (set! value_reg (* n value_reg))
      (set! pc apply-cont))))

(define fact-cps
  (lambda ()
    (if (= n_reg 0)
        (begin
          (set! value_reg 1)
          (set! pc apply-cont))
        (begin
          (set! k_reg (make-cont <cont-2> n_reg k_reg))
          (set! n_reg (- n_reg 1))
          (set! pc fact-cps)))))

;; top-level function
(define fact
  (lambda (n)
    (set! k_reg (make-cont <cont-1>))
    (set! n_reg n)
    (set! pc fact-cps)
    (trampoline)))

In [ ]:
(fact 5)